In [1]:
from tensorflow.keras.models import load_model
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from numpy import dot
from numpy.linalg import norm

In [2]:
weigths_path = '../pretrained/sketch_embedding.h5'
embedding_model = load_model(weigths_path)

In [4]:
datas = '/Users/chizhang/Downloads/archive/architectural-styles-dataset/'

In [20]:
data = {'class':[],'filepath':[]}
for subpath in os.listdir(datas):
    print(f'reading {datas}{subpath}')
    image_name = os.listdir(f'{datas}{subpath}')
    data['class'].extend([subpath]*len(image_name))
    data['filepath'].extend([f'{datas}{subpath}/{im}' for im in image_name])

reading /Users/chizhang/Downloads/archive/architectural-styles-dataset/Palladian architecture
reading /Users/chizhang/Downloads/archive/architectural-styles-dataset/Novelty architecture
reading /Users/chizhang/Downloads/archive/architectural-styles-dataset/International style
reading /Users/chizhang/Downloads/archive/architectural-styles-dataset/Chicago school architecture
reading /Users/chizhang/Downloads/archive/architectural-styles-dataset/Beaux-Arts architecture
reading /Users/chizhang/Downloads/archive/architectural-styles-dataset/Ancient Egyptian architecture
reading /Users/chizhang/Downloads/archive/architectural-styles-dataset/Gothic architecture
reading /Users/chizhang/Downloads/archive/architectural-styles-dataset/Tudor Revival architecture
reading /Users/chizhang/Downloads/archive/architectural-styles-dataset/Romanesque architecture
reading /Users/chizhang/Downloads/archive/architectural-styles-dataset/Colonial architecture
reading /Users/chizhang/Downloads/archive/architect

In [372]:
df = pd.DataFrame(data)

In [3]:
def sketchify(jc):

    jc = np.uint8(jc)
    
    scale_percent = 1

    width = int(jc.shape[1]*scale_percent)
    height = int(jc.shape[0]*scale_percent)

    dim = (width,height)
    resized = cv2.resize(jc,dim,interpolation = cv2.INTER_AREA)

    kernel_sharpening = np.array([[-1,-1,-1], 
                                  [-1, 9,-1],
                                  [-1,-1,-1]])
    sharpened = cv2.filter2D(resized,-1,kernel_sharpening)



    gray = cv2.cvtColor(sharpened , cv2.COLOR_BGR2GRAY)
    inv = 255-gray
    gauss = cv2.GaussianBlur(inv,ksize=(15,15),sigmaX=0,sigmaY=0)

    def dodgeV2(image,mask):
        return cv2.divide(image,255-mask,scale=256)

    pencil_jc = 255 - dodgeV2(gray,gauss)
    img = np.zeros(jc.shape)
    img[:,:,0] = pencil_jc
    img[:,:,1] = pencil_jc
    img[:,:,2] = pencil_jc
    return img

In [4]:
def get_embeedings(filepath):
    img = cv2.imread(filepath)
    img = cv2.resize(img,(256,256),interpolation = cv2.INTER_AREA)
    sketch_im = sketchify(img)/255
    embeddings = embedding_model.predict(sketch_im[np.newaxis,:,:,:])
    return embeddings[0]

In [375]:
from tqdm.notebook import tqdm

In [376]:
all_embeddings = []
file_list = df.filepath.to_list()
for i in tqdm(range(len(file_list))):
    filepath = file_list[i]
    all_embeddings.append(get_embeedings(filepath))

In [377]:
emb_df = pd.DataFrame(np.stack(all_embeddings))

df = pd.concat([df,emb_df],1)

df.to_csv('./ArchiFinder/embeddings.csv',index=False)